### Smooth Structured Hexahedral Meshing
#### Standalone execution

Created on: 12.05.2024

Last update: 12.05.2024

Author: Simone Poncioni, ARTORG Center for Biomedical Engineering Research, University of Bern, Bern, Switzerland

#### Aims

This example shows how to take advantage of the automatisation within ```pyhexspline``` to create smooth structured hexahedral meshes.

1. Load and inspect the cortical mask provided by SCANCO
2. Define settings and parameters
3. Generate the mesh
4. Visualize, save, evaluate mesh

#### Configuration and imports

In [ ]:
import logging
import os

import coloredlogs
import pyhexspline.futils.hfe_input_transformer as hit
from pyhexspline.spline_mesher import HexMesh

In [ ]:
# set up logging
LOGGING_NAME = "MESHING"
# configure the logger
logger = logging.getLogger(LOGGING_NAME)
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
handler.setFormatter(formatter)
logger.addHandler(handler)
coloredlogs.install(level=logging.INFO, logger=logger)


#### Define settings dictionary for ```pyhexspline```

In [ ]:
img_settings = {
    "img_basefilename": "C0003102",
    "img_basepath": f"{os.getcwd()}/01_AIM",
    "meshpath": f"{os.getcwd()}/03_MESH",
    "outputpath": f"{os.getcwd()}/04_OUTPUT",
}
meshing_settings = {
    "aspect": 100,                          # aspect ratio of the plots
    "_slice": 1,                            # slice of the image to be plotted
    "undersampling": 1,                     # undersampling factor of the image
    "slicing_coefficient": 10,              # using every nth slice of the image for the spline reconstruction
    "inside_val": int(0),                   # threshold value for the inside of the mask
    "outside_val": int(1),                  # threshold value for the outside of the mask
    "lower_thresh": float(0),               # lower threshold for the mask
    "upper_thresh": float(0.9),             # upper threshold for the mask
    "s": 10,                                # smoothing factor of the spline
    "k": 3,                                 # degree of the spline
    "interp_points": 350,                   # number of points to interpolate the spline
    "thickness_tol": 5e-1,                  # minimum cortical thickness tolerance: 3 * XCTII voxel size
    "phases": 2,                            # 1: only external contour, 2: external and internal contour
    "center_square_length_factor": 0.4,     # size ratio of the refinement square: 0 < l_f < 1
    "mesh_order": 1,                        # set order of the mesh (1: linear, 2: quadratic)****
    "n_elms_longitudinal": 5,               # number of elements in the longitudinal direction
    "n_elms_transverse_trab": 15,           # number of elements in the transverse direction for the trabecular compartment
    "n_elms_transverse_cort": 3,            # number of elements in the transverse direction for the cortical compartment
    "n_elms_radial": 15,                    # number of elements in the radial direction # ! Should be 10 if trab_refinement is True
    "ellipsoid_fitting": True,              # True: perform ellipsoid fitting
    "show_plots": False,                    # show plots during construction
    "show_gmsh": False,                      # show gmsh GUI
    "write_mesh": True,                     # write mesh to file
    "trab_refinement": False,               # True: refine trabecular mesh at the center
    "mesh_analysis": True,                  # True: perform mesh analysis (plot JAC det in GMSH GUI)
}

#### Read .MHD image of the CORTMASK and instantiate the HexMesh class

In [ ]:

sitk_image_s = sitk.ReadImage(
    "/home/simoneponcioni/Documents/01_PHD/03_Methods/HFE/01_DATA/454_L_94_F/C0003118_CORTMASK.mhd"
)

# Create the HexMesh object
mesh = HexMesh(
    meshing_settings,
    img_settings,
    sitk_image=sitk_image_s,
    logger=logging.getLogger(LOGGING_NAME),
)

#### Perform the meshing

In [ ]:
(
    nodes,
    elms,
    nb_nodes,
    centroids_cort,
    centroids_trab,
    elm_vol_cort,
    elm_vol_trab,
    radius_roi_cort,
    radius_roi_trab,
    bnds_bot,
    bnds_top,
    reference_point_coord,
) = mesh.mesher()

#### Print meshing information

In [ ]:
_w_text = 60  # width of the text
_w_value = 10  # width of the value
logger.info(f"{'Nodes:':<{_w_text}}{len(nodes):>{_w_value}}")
logger.info(f"{'Elements:':<{_w_text}}{len(elms):>{_w_value}}")
logger.info(f"{'Bottom boundary nodes:':<{_w_text}}{len(bnds_bot):>{_w_value}}")
logger.info(f"{'Top boundary nodes:':<{_w_text}}{len(bnds_top):>{_w_value}}")
logger.info(
    f"{'Centroids in trab physical group:':<{_w_text}}{len(centroids_trab):>{_w_value}}"
)
logger.info(
    f"{'Centroids in cort physical group:':<{_w_text}}{len(centroids_cort):>{_w_value}}"
)
logger.info(f"'Radius ROI cort: {radius_roi_cort:.3f} (mm)")
logger.info(f"'Radius ROI trab: {radius_roi_trab:.3f} (mm)")

elm_vol = np.concatenate((elm_vol_cort.flatten(), elm_vol_trab.flatten()))
min_volume = min(elm_vol)
if min_volume < 0:
    logger.critical(f"Negative volume detected: {min_volume:.3f} (mm^3)")
else:
    logger.info(f"Minimum element volume: {min_volume:.3f} (mm^3)")

print("-" * 150)